In [172]:
import pandas as pd
import joblib
from pathlib import Path
import os
import sys
import warnings
warnings.filterwarnings("ignore")
notebook_dir = os.path.dirname(os.path.abspath(''))
src_path = os.path.join(notebook_dir, '..\..\..\src')
sys.path.append(src_path)
print(src_path)

d:\ANNACONDA\projects\interest_rate_prediction\notebooks\data_cleaning\ecnomic_indicators\..\..\..\src


In [173]:
os.getcwd()

'd:\\ANNACONDA\\projects\\interest_rate_prediction\\notebooks\\data_cleaning\\ecnomic_indicators\\retail_sales'

In [174]:
pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 200)

In [175]:
from config import RETAIL_SALES_PATH

In [176]:
retail_excel = pd.ExcelFile(f"../../../{RETAIL_SALES_PATH}retail_sales.xlsx")
retail_excel.sheet_names

['2024',
 '2023',
 '2022',
 '2021',
 '2020',
 '2019',
 '2018',
 '2017',
 '2016',
 '2015',
 '2014',
 '2013',
 '2012',
 '2011',
 '2010',
 '2009',
 '2008',
 '2007',
 '2006',
 '2005',
 '2004',
 '2003',
 '2002',
 '2001',
 '2000',
 '1999',
 '1998',
 '1997',
 '1996',
 '1995',
 '1994',
 '1993',
 '1992']

In [ ]:
from datetime import datetime

In [275]:
def clean_sheets():
    merged = pd.DataFrame()
    for sheet in retail_excel.sheet_names:
        print(sheet)
        retail = pd.read_excel(f"../../../{RETAIL_SALES_PATH}retail_sales.xlsx", sheet_name=sheet)
        retail = retail.iloc[3:109, 1:]
        retail["Unnamed: 1"] = retail["Unnamed: 1"].fillna("Category")
        if "Unnamed: 14" in retail.columns:
            retail = retail.drop("Unnamed: 14", axis=1)
        else:
            retail = retail.drop(["Unnamed: 5", "Unnamed: 6"], axis=1)
        retail = retail.T.reset_index(drop=True)
        retail = retail.set_index(3)
        retail = retail.T
        retail_melted = retail.melt(id_vars=["Category"], var_name="Date", value_name="Unemployment_Rate")
        retail_melted = retail_melted.dropna()
        retail_melted["Date"] = retail_melted["Date"].apply(lambda x: str(x).split("(")[0])
        retail_melted["Date"] = pd.to_datetime(retail_melted["Date"])
        merged = pd.concat([merged, retail_melted], axis=0)
    merged["Year"] = merged["Date"].dt.year
    merged["Month"] = merged["Date"].dt.month
    merged = merged.drop("Date", axis=1)
    merged["Unemployment_Rate"] = pd.to_numeric(merged["Unemployment_Rate"], errors="coerce")
    merged["Year"] = merged["Year"].astype(str)
    merged["Month"] = merged["Month"].astype(str)
    numeric_columns = merged.select_dtypes(include=["number"]).columns.tolist()
    non_numeric_columns = merged.select_dtypes(exclude=["number"]).columns.tolist()
    group_cols = ["Year", "Month", "Category"]
    retail_agg = merged[group_cols + numeric_columns].groupby(group_cols, as_index=False).sum()
    retail_pivot = retail_agg.pivot_table(index=["Year", "Month"], columns="Category", values=numeric_columns, aggfunc="sum")
    retail_pivot = retail_pivot.reset_index()
    retail_pivot.columns.name = None
    

    return retail_pivot

In [276]:
retail_merged = clean_sheets()
retail_merged

2024
2023
2022
2021
2020
2019
2018
2017
2016
2015
2014
2013
2012
2011
2010
2009
2008
2007
2006
2005
2004
2003
2002
2001
2000
1999
1998
1997
1996
1995
1994
1993
1992


Year Month                 Unemployment_Rate  \
Category             All other gen. merchandise stores   
0         1992     1                            4793.0   
1         1992    10                            5142.0   
2         1992    11                            5459.0   
3         1992    12                            7081.0   
4         1992     2                            4770.0   
..         ...   ...                               ...   
382       2023     8                           17449.0   
383       2023     9                           17312.0   
384       2024     1                           17128.0   
385       2024     2                           17370.0   
386       2024     3                           18397.0   

                                            \
Category All other home furnishings stores   
0                                      0.0   
1                                      0.0   
2                                      0.0   
3                                      0.0   
4                                      0.0   
..                                     ...   
382                                    0.0   
383                                    0.0   
384                                    0.0   
385                                    0.0   
386                                    0.0   

                                                                        \
Category Automobile and other motor vehicle dealers Automobile dealers   
0                                           56955.0            25800.0   
1                                           65343.0            31578.0   
2                                           61244.0            27838.0   
3                                           62313.0            28237.0   
4                                           58660.0            27031.0   
..                                              ...                ...   
382                                        250570.0           120988.0   
383                                        244062.0           113720.0   
384                                        226794.0           102376.0   
385                                        238377.0           109399.0   
386                                        250911.0           119987.0   

                                                                               \
Category Automotive parts, acc., and tire stores Beer, wine and liquor stores   
0                                         6334.0                       1713.0   
1                                         7367.0                       1854.0   
2                                         7044.0                       1853.0   
3                                         7026.0                       1856.0   
4                                         6374.0                       1763.0   
..                                           ...                          ...   
382                                      23440.0                       5935.0   
383                                      22653.0                       6008.0   
384                                      21898.0                       5954.0   
385                                      21592.0                       6063.0   
386                                      22983.0                       6058.0   

                                                    \
Category Beer, wine, and liquor stores Book stores   
0                               1509.0       790.0   
1                               1882.0       645.0   
2                               1831.0       642.0   
3                               2511.0      1165.0   
4                               1541.0       539.0   
..                                 ...         ...   
382                             5947.0       878.0   
383                             5894.0       727.0   
384                             5007.0       802.0   
385                             5360.0       

In [277]:
path = Path(Path.cwd()/"../../../../data")
path

WindowsPath('d:/ANNACONDA/projects/interest_rate_prediction/notebooks/data_cleaning/ecnomic_indicators/retail_sales/../../../../data')

In [278]:
data_folder = path/'parquet/retail_sales'
data_folder.mkdir(parents=True, exist_ok=True)

In [280]:
retail_merged.to_parquet(data_folder/"retail_sales.parquet")